In [139]:
import pandas as pd
import numpy as np 
from scipy import stats

import statsmodels.stats.api as api
import statsmodels.formula.api as smf

Давайте проанализируем данные опроса 4361 женщин из Ботсваны:


О каждой из них мы знаем:

сколько детей она родила (признак ceb)

возраст (age)

длительность получения образования (educ)

религиозная принадлежность (religion)

идеальное, по её мнению, количество детей в семье (idlnchld)

была ли она когда-нибудь замужем (evermarr)

возраст первого замужества (agefm)

длительность получения образования мужем (heduc)

знает ли она о методах контрацепции (knowmeth)

использует ли она методы контрацепции (usemeth)

живёт ли она в городе (urban)

есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

Давайте научимся оценивать количество детей ceb по остальным признакам.
Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [140]:
df = pd.read_csv('botswana.tsv', sep='\t')

In [141]:
df.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [142]:
df['religion'].unique()

array(['catholic', 'protestant', 'spirit', 'other'], dtype=object)

Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?



In [143]:
df.dropna().shape

(1834, 15)

В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке $agefm$ пропуски стоят только там, где $evermarr=0$, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак $x_1$ на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

создать новый бинарный признак

$x_2=1$,если $x_1=$'не применимо'; 0,иначе;


заменить "не применимо" в $x_1$ на произвольную константу $c$, которая среди других значений $x_1$ не встречается.


Теперь, когда мы построим регрессию на оба признака и получим модель вида

$y=\beta_0 + \beta_1 x_1 + \beta_2 x_2$

на тех объектах, где $x_1$ было измерено, регрессионное уравнение примет вид

$y=\beta_0 + \beta_1 x$,

а там, где $x_1$ было "не применимо", получится

$y=\beta_0 + \beta_1 c + \beta_2$ 

Выбор $c$ влияет только на значение и интерпретацию $\beta_2$ , но не $\beta_1$ 

Давайте используем этот метод для обработки пропусков в agefm и heduc.

1. Создайте признак nevermarr, равный единице там, где в agefm пропуски.

2. Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице $X$ будет мультиколлинеарность.
3. Замените NaN в признаке agefm на $c_{agefm}=0$ 

4. У объектов, где nevermarr = 1, замените NaN в признаке heduc на $c_{heduc_1}=-1$ (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).

Сколько осталось пропущенных значений в признаке heduc?

In [144]:
#1
df['nevermarr'] = [1 if df.loc[i,'evermarr'] == 0 else 0 for i in range(df.shape[0])]

In [145]:
#2
df = df.drop(columns = ['evermarr'], axis=1)

In [146]:
#3
df.at[list(df[df['agefm'].isna() == True].index),'agefm'] = 0

In [147]:
#4
df.at[list(df[(df['nevermarr'] == 1)&(df['heduc'].isna() == True)].index), 'heduc'] = -1

In [148]:
df[df['heduc'].isna() == True].shape

(123, 15)

Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения ($c_{idlnchld}=-1, c_{heduc_2}=-2$ (значение $-1$ мы уже использовали), $c_{usemeth}=-1$).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [149]:
df['idlnchld_nans'] = 0
df.at[list(df[df['idlnchld'].isna() == True].index), 'idlnchld_nans'] = 1

df['heduc_nans'] = 0
df.at[list(df[df['heduc'].isna() == True].index), 'heduc_nans'] = 1

df['usemeth_nans'] = 0
df.at[list(df[df['usemeth'].isna() == True].index), 'usemeth_nans'] = 1

In [150]:
df.at[list(df[df['idlnchld'].isnull() == True].index), 'idlnchld'] = -1
df.at[list(df[df['heduc'].isnull() == True].index), 'heduc'] = -2
df.at[list(df[df['usemeth'].isnull() == True].index),'usemeth'] = -1

In [151]:
df = df.dropna()

In [152]:
df.shape[0]*df.shape[1]

78264

Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации $R^2$
 ? Округлите до трёх знаков после десятичной точки.

In [153]:
formula = 'ceb ~'+'+'.join(df.columns[1:])

In [154]:
regression = smf.ols(formula, data=df)

print(regression.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Mon, 30 Sep 2019   Prob (F-statistic):               0.00
Time:                        23:50:19   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1.

In [155]:
print('критерий Бройша-Пагана: %f'%api.het_breuschpagan(regression.fit().resid, regression.fit().model.exog)[1])

критерий Бройша-Пагана: 0.000000


In [156]:
regression_fixed = smf.ols(formula, data=df)

print(regression_fixed.fit(cov_type='HC1').summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Mon, 30 Sep 2019   Prob (F-statistic):               0.00
Time:                        23:50:21   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [158]:
df_ = df.drop(columns = ['religion', 'radio', 'tv'], axis = 1)

In [164]:
formula_ = 'ceb~'+'+'.join(df_.columns[1:])
regression = smf.ols(formula_, data=df_)
regr_f = regression.fit()
print('критерий Бройша-Пагана: %f'%api.het_breuschpagan(regr_f.resid, regr_f.model.exog)[1])

критерий Бройша-Пагана: 0.000000


In [165]:
regressionnn = smf.ols(formula_, data=df_)
modell = regressionnn.fit(cov_type='HC1')
print(modell.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Mon, 30 Sep 2019   Prob (F-statistic):               0.00
Time:                        23:56:14   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.0698      0.258     -4.152

In [170]:
print(regression.fit().compare_f_test(regressionnn.fit()))

(nan, nan, 0.0)


/home/danone/anaconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:2026: RuntimeWarning: invalid value encountered in double_scalars
  f_value = (ssr_restr - ssr_full) / df_diff / ssr_full * df_full


Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением $c_{usemeth }=-1$, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили $5.5\times10^{-8}$,нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans.

In [173]:
formulla = 'ceb~'+'age+educ+religion+idlnchld+knowmeth+agefm+heduc+urban+electric+radio+tv+bicycle+nevermarr+idlnchld_nans+heduc_nans'
formulla

'ceb~age+educ+religion+idlnchld+knowmeth+agefm+heduc+urban+electric+radio+tv+bicycle+nevermarr+idlnchld_nans+heduc_nans'

In [174]:
regul = smf.ols(formulla, data=df)
modeL = regul.fit()
print(modeL.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     432.2
Date:                Tue, 01 Oct 2019   Prob (F-statistic):               0.00
Time:                        00:04:54   Log-Likelihood:                -7822.1
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4330   BIC:                         1.579e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [175]:
print('F=%f, p=%.40f, k1=%f' % regressionnn.fit().compare_f_test(regul.fit()))

F=-59.384822, p=nan, k1=-3.000000


In [177]:
print(modell.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Tue, 01 Oct 2019   Prob (F-statistic):               0.00
Time:                        00:07:09   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.0698      0.258     -4.152